In [1]:
import openai
import json


def generate_lessons_data(language, level, topic, N_runs = 5, lesson_kinds=['conversations', 'pairs', 'translations']):
    def make_conversation_system(language, topic, level):
        return {"role": "system", "content": f'You are a language learning tool, your task is to generate produce JSON containing a series of {language} conversations tailored to what you would expect in a {level} level text book on "{topic}" in {language} at {level} level.'}

    def make_pairs_system(language, topic, level):
        return {"role": "system", "content": f'You are a language learning tool, your task is to generate produce JSON containing a series of pairs of words in {language} and their translations in json, each pair should be a common word at {level} level and varied, these words should be related to the topic of "{topic}".'}

    def make_translation_system(language, topic, level):
        return {"role": "system", "content": f'You are a language learning tool, your task is to generate {language} sentences tailored to what you would expect in a {level} level text book and their English translations. Responses should be in JSON and be focused on the topic of "{topic}".'}

    conversation_assistant = {
        "role": "assistant",
        "content": '[{"conversation": [{"speaker": "person1", "message": "Quel est ton animal préféré ?"}, {"speaker": "person2", "message": "Mon animal préféré est le chat. Et toi ?"}, {"speaker": "person1", "message": "J\'aime les chiens. Ils sont très fidèles."}], "conversation_summary": "A conversation discussing each person\'s favorite animal."}, {"conversation": [{"speaker": "person1", "message": "Tu aimes la nature ?"}, {"speaker": "person2", "message": "Oui, j\'adore me promener dans la forêt."}, {"speaker": "person1", "message": "Moi aussi. C\'est très relaxant d\'écouter les oiseaux."}], "conversation_summary": "A conversation about enjoying nature, especially walking in the forest and listening to birds."}]'
    }

    pairs_assistant = {
        "role": "assistant",
        "content": '[{"English": "apple", "French": "pomme"}, {"English": "book", "French": "livre"}, {"English": "car", "French": "voiture"}, {"English": "dog", "French": "chien"}, {"English": "house", "French": "maison"}, {"English": "love", "French": "amour"}, {"English": "moon", "French": "lune"}, {"English": "sun", "French": "soleil"}, {"English": "tree", "French": "arbre"}, {"English": "water", "French": "eau"}]'
    }

    translation_assistant = {
        "role": "assistant",
        "content": '[{"English": "How are you today?", "French": "Comment vas-tu aujourd\'hui ?"}, {"English": "I am learning to speak French.", "French": "J\'apprends à parler français."}, {"English": "The weather is nice this afternoon.", "French": "Le temps est agréable cet après-midi."}, {"English": "Can you help me with my homework?", "French": "Peux-tu m\'aider avec mes devoirs ?"}, {"English": "My family is going on vacation next week.", "French": "Ma famille part en vacances la semaine prochaine."}, {"English": "This restaurant has delicious food.", "French": "Ce restaurant a de la nourriture délicieuse."}]'
    }

    client = openai.OpenAI(
        base_url="http://localhost:8080/v1",  # "http://<Your api-server IP>:port"
        api_key="sk-no-key-required"
    )

    user1 = {"role": "user", "content": "Generate some language exercises for me."}
    user2 = {"role": "user", "content": "Create 5 more examples in the same JSON format."}      

    def generate_responses():
        for run in range(N_runs):
            conversation_system = make_conversation_system(language, topic, level)
            pairs_system = make_pairs_system(language, topic, level)
            translation_system = make_translation_system(language, topic, level)
            systems = []
            assistants = []

            if 'conversations' in lesson_kinds:
                systems.append(conversation_system)
                assistants.append(conversation_assistant)
            if 'pairs' in lesson_kinds:
                systems.append(pairs_system)
                assistants.append(pairs_assistant)
            if 'translations' in lesson_kinds: 
                systems.append(translation_system)
                assistants.append(translation_assistant)


            for idx, (system, assistant) in enumerate(zip(systems, assistants)):
                completion = client.chat.completions.create(
                    model="mistral",
                    messages=[
                        system,
                        user1,
                        assistant,
                        user2
                    ],
                    temperature=0.8,
                )
                raw_response = completion.choices[0].message.content
                yield raw_response

    return generate_responses()

def dump_response(response, language, topic, level, idx, run):
    try:
        response = json.loads(response)
        # write to file
        with open(f"{language}_{topic}_{level}_{idx}_{run}.json", "w") as f:
            json.dump(response, f, indent=4, ensure_ascii=False)
    except json.JSONDecodeError:
        # write as text
        with open(f"{language}_{topic}_{level}_{idx}_broken_{run}.json", "w") as f:
            f.write(response)


In [2]:

topics = ["Animals and Nature", "Family", "Travel", "Food and Drink", "Weather", "Shopping", "Health", "Work", "Sports", "Culture"]
french_food_beginner_conversations_f = generate_lessons_data("French", "beginner", "Food and Drink", N_runs=5, lesson_kinds=['conversations'])

In [3]:
first_lesson = next(french_food_beginner_conversations_f)
print(first_lesson)

[{ "conversation": [{"speaker": "person1", "message": "Qu'est-ce que tu manges pour déjeuner ?"}, {"speaker": "person2", "message": "Je mange un sandwich au fromage et aux légumes."}, {"speaker": "person1", "message": "C'est une bonne idée ! J'aime les sandwiches au fromage."}], "conversation_summary": "A conversation about what to eat for lunch."}, {"conversation": [{"speaker": "person1", "message": "J'adore les desserts ! Qu'est-ce que tu préfères ?"}, {"speaker": "person2", "message": "Je préfère le gâteau au chocolat."}, {"speaker": "person1", "message": "C'est délicieux ! Je préfère le tarte aux fruits."}], "conversation_summary": "A conversation about favorite desserts."}, {"conversation": [{"speaker": "person1", "message": "Aimes-tu le thé ?"}, {"speaker": "person2", "message": "Oui, j'en bois souvent le soir."}, {"speaker": "person1", "message": "Moi aussi. J'adore le thé avec du sucre et de la crème."}], "conversation_summary": "A conversation about liking tea."}, {"conversati

In [6]:
first_lesson.replace("<|eot_id|>", "")

'[{ "conversation": [{"speaker": "person1", "message": "Qu\'est-ce que tu manges pour déjeuner ?"}, {"speaker": "person2", "message": "Je mange un sandwich au fromage et aux légumes."}, {"speaker": "person1", "message": "C\'est une bonne idée ! J\'aime les sandwiches au fromage."}], "conversation_summary": "A conversation about what to eat for lunch."}, {"conversation": [{"speaker": "person1", "message": "J\'adore les desserts ! Qu\'est-ce que tu préfères ?"}, {"speaker": "person2", "message": "Je préfère le gâteau au chocolat."}, {"speaker": "person1", "message": "C\'est délicieux ! Je préfère le tarte aux fruits."}], "conversation_summary": "A conversation about favorite desserts."}, {"conversation": [{"speaker": "person1", "message": "Aimes-tu le thé ?"}, {"speaker": "person2", "message": "Oui, j\'en bois souvent le soir."}, {"speaker": "person1", "message": "Moi aussi. J\'adore le thé avec du sucre et de la crème."}], "conversation_summary": "A conversation about liking tea."}, {"c

In [4]:
dump_response(first_lesson, "French", "Food and Drink", "beginner", 0, 0)